In [1]:
import os
import sys
import numpy as np
import mxnet as mx
from mxnet import gluon, nd
from collections import namedtuple
from common.params_inf import *
from common.utils import *

In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [4]:
Batch = namedtuple('Batch', ['data'])
ctx = mx.gpu(0)

In [5]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCH_SIZE*BATCHES_GPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(1280, 224, 224, 3) (1280, 3, 224, 224)


In [6]:
# Download Resnet weights
path='http://data.mxnet.io/models/imagenet/'
mx.test_utils.download(path+'resnet/50-layers/resnet-50-symbol.json')
mx.test_utils.download(path+'resnet/50-layers/resnet-50-0000.params')
print("Downloaded")

Downloaded


In [7]:
# Load model
sym, arg_params, aux_params = mx.model.load_checkpoint('resnet-50', 0)
# List the last 10 layers
all_layers = sym.get_internals()

In [8]:
# Get last layer
flatten_layer = all_layers['flatten0_output']

In [9]:
def predict_fn(classifier, data, batchsize, ctx):
    """ Return features from classifier """
    out = nd.zeros((len(data), RESNET_FEATURES), dtype=np.float32, ctx=ctx)    
    for idx, dta in yield_mb_X(data, batchsize):
        classifier.forward(Batch(data=[mx.nd.array(dta)]))
        out[idx*batchsize:(idx+1)*batchsize] = classifier.get_outputs()[0]
    nd.waitall()
    return out

In [10]:
# Get last layer
fe_sym = all_layers['flatten0_output']
# Initialise GPU
fe_mod = mx.mod.Module(symbol=fe_sym, context=ctx, label_names=None)
fe_mod.bind(for_training=False, inputs_need_grad=False,
            data_shapes=[('data', (BATCH_SIZE,3,224,224))])
fe_mod.set_params(arg_params, aux_params)

In [11]:
cold_start = predict_fn(fe_mod, fake_input_data_cf, BATCH_SIZE, ctx)

In [12]:
%%time
features = predict_fn(fe_mod, fake_input_data_cf, BATCH_SIZE, ctx)

CPU times: user 2 s, sys: 631 ms, total: 2.63 s
Wall time: 2.1 s


In [14]:
print("Images per second {}".format((BATCH_SIZE*BATCHES_GPU)/2.1))

Images per second 609.5238095238095
